In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL
import numerical_utils as nuti

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import time
import copy
import cftime

importlib.reload( LL )
importlib.reload( GrU)
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
uti.MakeDict4Exp( help=True)

In [ ]:
# Issue 24: 
#   last run I can find that used POP 
#   b.e21.BWsc1850.ne30_L48_BL10_cesm2_3_alpha05c_cam6_3_028_cam6_parcel_zm.004_zm2.hf
#   in /glade/campaign/cgd/amp/amwg/runs/
# 
# Configuration "26"
#   b.cesm3_cam058_mom_e.B1850WscMOM.ne30_L58_t061.camdev_cice5.026g
#   in /glade/campaign/cesm/development/omwg/projects/MOM6

config26 ='b.cesm3_cam058_mom_e.B1850WscMOM.ne30_L58_t061.camdev_cice5.026g' 
last_pop ='b.e21.BWsc1850.ne30_L48_BL10_cesm2_3_alpha05c_cam6_3_028_cam6_parcel_zm.004_zm2.hf'
B = uti.MakeDict4Exp( exp=config26 , user='omwg_mom6', subd='hist' , hsPat='cam.h0' , ymdPat='0001*' ,verbose=True, open_dataset=True )
A = uti.MakeDict4Exp( exp=last_pop , user='amwg_runs', subd='hist' , hsPat='cam.h0' , ymdPat='0001*' ,verbose=True, open_dataset=True )



In [ ]:

A.X['PRECT'] = A.X['PRECC'] + A.X['PRECL'] 
B.X['PRECT'] = B.X['PRECC'] + B.X['PRECL'] 


In [ ]:
fld='PRECT'
season='jan'
#UUc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='U',return_time=True)
pcp_a_s,years_a,months_a = Av.Seasonal( ds=A.X , season=season , fld=fld ,return_time=True)
pcp_b_s,years_b,months_b = Av.Seasonal( ds=B.X , season=season , fld=fld ,return_time=True)


In [ ]:
fld='TS'

#UUc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='U',return_time=True)
ts_a_s,years_x,months_x = Av.Seasonal( ds=A.X , season=season , fld=fld ,return_time=True)
ts_b_s,years_x,months_x = Av.Seasonal( ds=B.X , season=season , fld=fld ,return_time=True)


In [ ]:
fld='SWCF'

#UUc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='U',return_time=True)
swcf_a_s,years_x,months_x = Av.Seasonal( ds=A.X , season=season , fld=fld ,return_time=True)
swcf_b_s,years_x,months_x = Av.Seasonal( ds=B.X , season=season , fld=fld ,return_time=True)


In [ ]:

lon=A.X.lon.values
lat=A.X.lat.values

lapts_a_s = nuti.Sphere_Lap2( f =ts_a_s , lon=lon,lat=lat,wrap=True )
lapts_b_s = nuti.Sphere_Lap2( f =ts_b_s , lon=lon,lat=lat,wrap=True )


In [ ]:


landf=A.X.LANDFRAC[0,:,:].values
print( np.shape(landf))

lapts_a_s[0:5,:]=0.
lapts_a_s[-5:-1,:]=0.
lapts_b_s[0:5,:]=0.
lapts_b_s[-5:-1,:]=0.

lapts_a_s_o = np.where( landf > 0.01 , 0., lapts_a_s )
lapts_b_s_o = np.where( landf > 0.01 , 0., lapts_b_s )

lapts_a_s_o[0:25,:]=0.
lapts_a_s_o[-25:,:]=0.
lapts_b_s_o[0:25,:]=0.
lapts_b_s_o[-25:,:]=0.


In [ ]:
"""
from scipy.signal import convolve2d


# 1-2-1 smoothing kernel
kernel = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16

# Apply the 1-2-1 smoother
smoothed_lapts_a_s_o = convolve2d(lapts_a_s_o, kernel, mode='same', boundary='wrap')
"""

In [ ]:

pm_a,pi_a,dp_a = MkP.Pressure ( am=A.X.hyam.values , 
                               bm=A.X.hybm.values , 
                               ai=A.X.hyai.values , 
                               bi=A.X.hybi.values , 
                               ps=A.X.PS.values  , 
                               p_00=100_000., Gridkey='tzyx' )

pm_b,pi_b,dp_b = MkP.Pressure ( am=B.X.hyam.values , 
                               bm=B.X.hybm.values , 
                               ai=B.X.hyai.values , 
                               bi=B.X.hybi.values , 
                               ps=B.X.PS.values  , 
                               p_00=100_000., Gridkey='tzyx' )

A.X['UDP'] = dp_a * A.X.U
A.X['VDP'] = dp_a * A.X.V

B.X['UDP'] = dp_b * B.X.U
B.X['VDP'] = dp_b * B.X.V



In [ ]:
fld='UDP'

#UUc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='U',return_time=True)
udp_a_s,years_x,months_x = Av.Seasonal( ds=A.X , season=season , fld=fld ,return_time=True)
udp_b_s,years_x,months_x = Av.Seasonal( ds=B.X , season=season , fld=fld ,return_time=True)
fld='VDP'

#UUc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='U',return_time=True)
vdp_a_s,years_x,months_x = Av.Seasonal( ds=A.X , season=season , fld=fld ,return_time=True)
vdp_b_s,years_x,months_x = Av.Seasonal( ds=B.X , season=season , fld=fld ,return_time=True)


In [ ]:
divp_a_s = nuti.Sphere_Div2( f_x =udp_a_s[57,:,:] ,f_y =vdp_a_s[57,:,:] , lon=lon,lat=lat,wrap=True )
divp_b_s = nuti.Sphere_Div2( f_x =udp_b_s[57,:,:] ,f_y =vdp_b_s[57,:,:] , lon=lon,lat=lat,wrap=True )
divp_a_s[0:25,:]=0.
divp_a_s[-25:,:]=0.
divp_b_s[0:25,:]=0.
divp_b_s[-25:,:]=0.


In [ ]:
print(r'$\nabla^2 T_s$')

In [ ]:

divlev= np.linspace( -0.012,0.012, num=25 )
divscale=1.

laptlev=0.5*np.linspace( -1,1, num=31 )
laptscale=1.e10

lons=[B.X.lon]
lats=[B.X.lat] 
cmaps = ['bwr']

prclev=np.linspace( 0,20, num=21 )
prcscale=1000.*86400.

#fields=[ scale*aa_s , scale*bb_s  ]

tslev=np.linspace( 270,300, num=31 )
tsscale=1.
swcflev=np.linspace( -180,0, num=19 )
swcfscale=1.

titles = [ LL.titleGen( fld='PRECT',exp='POP',season=season,years=years_a) , 
           LL.titleGen( fld='PRECT',exp='MOM',season=season,years=years_b) , 
         LL.titleGen( fld='Div',exp='POP',season=season,years=years_a) , 
         LL.titleGen( fld='Div',exp='MOM',season=season,years=years_b) , 
         LL.titleGen( fld='TS',exp='POP',season=season,years=years_a) , 
         LL.titleGen( fld='TS',exp='MOM',season=season,years=years_b) , 
         LL.titleGen( fld='SWCF',exp='POP',season=season,years=years_a) , 
         LL.titleGen( fld='SWCF',exp='MOM',season=season,years=years_b) , ]
#         LL.titleGen( fld=r'$\nabla^2 T_s$',exp='POP',season=season,years=years_a) , 
#         LL.titleGen( fld=r'$\nabla^2 T_s$',exp='MOM',season=season,years=years_b) , ]


fields=[   prcscale*pcp_a_s , prcscale*pcp_b_s,  
        divscale*divp_a_s , divscale*divp_b_s, 
        tsscale*ts_a_s , tsscale*ts_b_s ,
        swcfscale*swcf_a_s , swcfscale*swcf_b_s   ]
        #laptscale*lapts_a_s_o , laptscale*lapts_b_s_o   ]

clevs=[ prclev, prclev, divlev , divlev, tslev, tslev, swcflev, swcflev ] #laptlev, laptlev ]
cmaps = [ 'terrain_r','terrain_r',  'bwr'  ,  'bwr'  , 'rainbow','rainbow', 'gist_ncar', 'gist_ncar' ,] #  'bwr'  ,  'bwr'  ,]

LL.Maps( fields=fields ,lons=lons ,lats=lats ,clevs=clevs,cmaps=cmaps, titles=titles, Arrangement=(2,4) )


In [ ]:
fig=plt.subplots()
title_ = LL.titleGen( fld='SST v SWCF (60S-10S)',exp='MOM-POP',season=season,years=years_b)
lon_grid, lat_grid = np.meshgrid(lon, lat)
oo=np.where(   (lat_grid.flatten() > -60.) & (lat_grid.flatten() < -10.) & (landf.flatten() < 0.01 ) )
print( lat[90])
plt.scatter( ts_a_s.flatten()[oo[0]] , swcf_a_s.flatten()[oo[0]] , s=.5 , label='Last POP run')
plt.scatter( ts_b_s.flatten()[oo[0]] , swcf_b_s.flatten()[oo[0]] , s=.5 , label="'26' (MOM)")
plt.xlabel( "SST (K)" ) 
plt.ylabel( "SWCF "+r'($\mathrm{Wm^{-2}}$)' ) 
plt.title( title_ )
plt.legend(markerscale=5)



In [ ]:

clev=np.linspace( 270,300, num=31 )
scale=1.

fields=[ scale*ts_a_s , scale*ts_b_s  ]
lons=[B.X.lon]
lats=[B.X.lat] 
cmaps = ['terrain_r','terrain_r']
clevs=[clev,clev]

LL.Maps( fields=fields ,lons=lons ,lats=lats ,clevs=clevs,cmaps=cmaps )
